<a href="https://colab.research.google.com/github/wesslen/llm-evaluations/blob/main/notebooks/02_rag_deepeval_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 RAG/LLM Evaluators - DeepEval

This code tutorial shows how you can easily integrate DeepEval with LlamaIndex. DeepEval makes it easy to unit-test your RAG/LLMs.

You can read more about the DeepEval framework here: https://docs.confident-ai.com/docs/getting-started

Feel free to check out our repository here on GitHub: https://github.com/confident-ai/deepeval

### Set-up and Installation

We recommend setting up and installing via pip!

In [1]:
%%capture
# use uv for virtual environment https://docs.astral.sh/uv/
!uv pip install --system llama-index deepeval

Deepeval has a fully hosted option. We'll ignore this and keep everything local (granted on Colab cloud).

In [2]:
#!deepeval login

## Types of Metrics

DeepEval presents an opinionated framework for unit testing RAG applications. It breaks down evaluations into test cases, and offers a range of evaluation metrics that you can freely evaluate for each test case, including:

- G-Eval
- Summarization
- Answer Relevancy
- Faithfulness
- Contextual Recall
- Contextual Precision
- Contextual Relevancy
- RAGAS
- Hallucination
- Bias
- Toxicity

[DeepEval](https://github.com/confident-ai/deepeval) incorporates the latest research into its evaluation metrics, which are then used to power LlamaIndex's evaluators. You can learn more about the full list of metrics and how they are calculated [here.](https://docs.confident-ai.com/docs/metrics-introduction)

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

> ⚠ **How to use Open LLMs like Llama 3**: Many Open LLMs are deployed on endpoints that mimic OpenAI's endpoints (namely, its chat completions). This is important as it enables connection by updating the host URL (`base_url`), pointing the client to your personal endpoint, not OpenAI's servers. You will also need to pass an API key. One of the most popular inference engines (vLLM) that implements [OpenAI compatability](https://docs.vllm.ai/en/v0.6.0/serving/openai_compatible_server.html).



## Step 1 - Setting Up LlamaIndex

In [4]:
# r.jina.ai converts pdf to markdown
# take Wells Fargo code of conduct
!wget -O text.txt https://r.jina.ai/https://www.wellsfargo.com/assets/pdf/about/corporate/code-of-conduct.pdf

--2024-11-11 23:17:42--  https://r.jina.ai/https://www.wellsfargo.com/assets/pdf/about/corporate/code-of-conduct.pdf
Resolving r.jina.ai (r.jina.ai)... 104.26.11.242, 104.26.10.242, 172.67.70.54, ...
Connecting to r.jina.ai (r.jina.ai)|104.26.11.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41541 (41K) [text/plain]
Saving to: ‘text.txt’

text.txt            100%[===================>]  40.57K  --.-KB/s    in 0.004s  

2024-11-11 23:17:42 (9.15 MB/s) - ‘text.txt’ saved [41541/41541]



In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

# Loading text files in root
documents = SimpleDirectoryReader(".").load_data()

Settings.chunk_size = 100
Settings.chunk_overlap = 20

index = VectorStoreIndex.from_documents(documents)
rag_application = index.as_query_engine(similarity_top_k=2)

## Step 2 - Using DeepEval's RAG/LLM evaluators

DeepEval offers 6 evaluators out of the box, some for RAG, some directly for LLM outputs (although also works for RAG). Let's follow the [RAG Triad docs](https://docs.confident-ai.com/docs/guides-rag-triad) to evaluate RAG like:

![](https://d2lsxfc3p6r9rv.cloudfront.net/rag-triad.svg)

In [6]:
from deepeval.test_case import LLMTestCase

# An example input to your RAG application
user_input = "Under what circumstances can an employee use their \
professional designation for a medical license while working at \
Wells Fargo, and what are the limitations?"

# LlamaIndex returns a response object that contains
# both the output string and retrieved nodes
response_object = rag_application.query(user_input)

# Process the response object to get the output string
# and retrieved nodes
if response_object is not None:
    actual_output = response_object.response
    retrieval_context = [node.get_content() for node in response_object.source_nodes]

# Create a test case and metric as usual
test_case = LLMTestCase(
    input=user_input,
    actual_output=actual_output,
    retrieval_context=retrieval_context
)

print(f"User input: {user_input}")

User input: Under what circumstances can an employee use their professional designation for a medical license while working at Wells Fargo, and what are the limitations?


In [7]:
print(f"Response: {response_object}")

Response: An employee at Wells Fargo can use their medical license professional designation if it is appropriate for their role. However, they must not misrepresent or use their professional designation if it is not suitable for their position or if it is prohibited by company policy or applicable laws and regulations.


In [8]:
print(f"Retrieval context: {retrieval_context}")

Retrieval context: ['12 of 19 Code of Conduct \n\n# Examples of potential conflicts, continued \n\nUse of professional designations \n\nWells Fargo acknowledges employees may maintain specialized, professional designations that may not relate to their duties with the company. These include but are not limited to legal, medical, notary, accounting, and investment licenses and certifications.', 'Employees must not misrepresent or use their professional designation if it is not appropriate for their role or if prohibited by company policy or applicable laws and regulations. \n\nFiduciary and investment duties \n\nWhen executing fiduciary duties or responsibilities, acting as a trustee, investment manager, or in any similar capacity in which the company possesses investment discretion on behalf of another, Wells Fargo acts in the best interest of our clients.']


In [9]:
# Evaluate
from deepeval import evaluate
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric, ContextualRelevancyMetric

faithfulness = FaithfulnessMetric()
answer_relevancy = AnswerRelevancyMetric()
contextual_relevancy = ContextualRelevancyMetric()

evaluate(test_cases=[test_case], metrics=[answer_relevancy, faithfulness, contextual_relevancy])

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:08,  8.57s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the answer is perfectly relevant and addresses the input directly without any irrelevant statements. Great job!, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. Great job maintaining accuracy and alignment!, error: None)
  - ❌ Contextual Relevancy (score: 0.2, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.20 because the context provides a general policy on using professional designations but lacks specific information on using a medical license at Wells Fargo, as noted in the reasons for irrelevancy and the relevant statement., error: None)

For test case:

  - input: Under what circumstances can an employee use their professional designation 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

EvaluationResult(test_results=[TestResult(success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the answer is perfectly relevant and addresses the input directly without any irrelevant statements. Great job!', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.003725, verbose_logs='Statements:\n[\n    "An employee at Wells Fargo can use their medical license professional designation if it is appropriate for their role.",\n    "They must not misrepresent or use their professional designation if it is not suitable for their position.",\n    "It is prohibited by company policy or applicable laws and regulations."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": null\n    }\n]'), MetricData(name='Faithfulness', threshold=0.5, s

Try these challenging questions:

### Multi-hop

Requires combining multiple conditions from the volunteer activities section:

> If an employee wants to volunteer at a nonprofit organization's board AND manage their investments AND receive compensation, what specific pre-clearance requirements apply?

Combines MNPI rules with hedging/derivatives restrictions:

> If an employee receives material nonpublic information about a company AND owns derivatives in that company's stock through a previously approved trading plan, what actions must they take?

### Complex

Requires combining personal relationships, fiduciary duties, and customer relationship contexts

> Can a Wells Fargo employee accept a position as a trustee for their cousin's estate if the cousin was also a Wells Fargo customer who they previously helped with banking services?

Check out [this RAG Evaluation guide](https://docs.confident-ai.com/docs/guides-rag-evaluation) on Deepeval's docs.

## Unit Testing

Let's ask a critical question for compliance on gifts and entertainment:

> Is it permissible for an employee to accept a cash gift from a customer as a thank you for exceptional service?


In [10]:
## save this string as a test_example.py

string_content = """
import pytest
from deepeval import assert_test
from deepeval.metrics import FaithfulnessMetric
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset, Golden

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

documents = SimpleDirectoryReader(".").load_data()

Settings.chunk_size = 100
Settings.chunk_overlap = 20

index = VectorStoreIndex.from_documents(documents)
rag_application = index.as_query_engine(similarity_top_k=2)

example_golden = Golden(input="Is it permissible for an employee to accept a cash gift from a customer as a thank you for exceptional service?")

dataset = EvaluationDataset(goldens=[example_golden])

@pytest.mark.parametrize(
    "golden",
    dataset.goldens,
)
def test_rag(golden: Golden):
    # LlamaIndex returns a response object that contains
    # both the output string and retrieved nodes
    response_object = rag_application.query(golden.input)

    # Process the response object to get the output string
    # and retrieved nodes
    if response_object is not None:
        actual_output = response_object.response
        retrieval_context = [node.get_content() for node in response_object.source_nodes]

    test_case = LLMTestCase(
        input=golden.input,
        actual_output=actual_output,
        retrieval_context=retrieval_context
    )
    print("\\n")
    print(f"Query: {test_case.input}")
    print(f"Response: {test_case.actual_output}")
    print(f"Retrieval context: {test_case.retrieval_context}")
    answer_faithfulness_metric = FaithfulnessMetric(threshold=0.5)
    answer_faithfulness_metric.measure(test_case)
    print(f"Metric score: {answer_faithfulness_metric.score}")
    print(f"Metric reason: {answer_faithfulness_metric.reason}")
    assert_test(test_case, [answer_faithfulness_metric])
"""

with open("test_example.py", "w") as file:
    file.write(string_content)

print("String saved to test_example.py")

String saved to test_example.py


In [11]:
!pytest -s test_example.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0
rootdir: /content
plugins: repeat-0.9.3, deepeval-1.5.0, xdist-3.6.1, typeguard-4.4.1, anyio-3.7.1
collected 1 item                                                                                   

test_example.py 

Query: Is it permissible for an employee to accept a cash gift from a customer as a thank you for exceptional service?
Response: It is not permissible for an employee to accept a cash gift from a customer as a thank you for exceptional service.
Retrieval context: ['Employees should conduct themselves in accordance with the following expectations: \n\n• Refrain from giving and receiving gifts offered in exchange for business referrals or other business advantages. \n\n• Never give or receive gifts that are cash or cash equivalents, cannabis-related, or otherwise do not comply with our policies. \n\n• Follow requir

Examples of other unit tests to improve risk management:

### Compliance

> Can an employee delay reporting a potential violation of the Code of Conduct if they're waiting to gather more evidence?

### Information security

> Is it acceptable for an employee to use their Wells Fargo credentials to access customer information to help a family member evaluate a business opportunity?

### Reputational risk

> Can an employee make anonymous posts on social media criticizing Wells Fargo's business practices if they believe the criticism is justified?

### Ethical conduct

> If an employee discovers a colleague is manipulating records to meet sales targets but the manipulation benefits the company, should they report it?

## Full List of Evaluators

Here is how you can import all 6 metrics from `deepeval`:

```python
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualRelevancyMetric,
    SummarizationMetric,
    BiasMetric,
    ToxicityMetric,
)
```

For all evaluator definitions and to understand how it integrates with DeepEval's testing suite, [click here.](https://docs.confident-ai.com/docs/integrations-llamaindex)

## Useful Links

- [DeepEval Quickstart](https://docs.confident-ai.com/docs/getting-started)
- [Everything you need to know about LLM evaluation metrics](https://www.confident-ai.com/blog/llm-evaluation-metrics-everything-you-need-for-llm-evaluation)